In [ ]:
import nltk
import pandas as pd
import random
import numpy
import string
import math

from numpy import array
from nltk.corpus import stopwords
from collections import Counter
from scipy.sparse import csr_matrix
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
from gensim import corpora, models, utils

In [ ]:
#TODO: Lemmatize correctly, this lemmatizer is really basic, it doesn't recognize verbs and nouns
sw = [x.replace('\n', '') for x in open('stopwords-fr.txt', mode="r", encoding="utf-8").readlines()] + stopwords.words('french') + stopwords.words('english')
lemmatizer = FrenchLefffLemmatizer()
stemmer = nltk.stem.snowball.FrenchStemmer()

#stem, lemmatize, remove punctuation, and return a string
def process(article):
    res = ""
    for word in word_tokenize(article):
        if not word in sw+string.punctuation.split():
            if res == "": #if first word
                res += stemmer.stem(lemmatizer.lemmatize(word.lower()))
            else:
                res += " " + stemmer.stem(lemmatizer.lemmatize(word.lower()))
    return res


In [ ]:
data = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t', header=0)

#Let's process our corpus, and determine a limit to split it in partitions
limit = 0
usable = []
for i in range(0, len(data['abstract']), 1):
    if not isinstance(data['abstract'][i], float): #if not null, empty, or whatever (so if there is a abstract)
        usable.append(process(data['abstract'][i]))
        if data['year'][i] <= 2010:
            limit += 1
    
print("nombre d'articles =", len(usable))
print("limit =", limit)

usable

In [ ]:
#TfIdf

vectorizer = TfidfVectorizer(stop_words=sw, use_idf=True, ngram_range=(1,3))
tfidf = vectorizer.fit_transform(usable)

tfidf

In [ ]:
#This was a try to use gensim. However, I don't success to use it (not yet! >:)c )

nb_concepts = 30
lsi = LsiModel(tfidf, num_topics=nb_concepts)

lsi.print_topics(num_topics=2, num_words=5)

#### Nous obtenons ici x clusters, représentés par des concepts grâce au LSA
#### Nous allons seuiller et étiquetter ces clusters

In [ ]:
#Here, I determine a tresh, to remove documents from clusters
#Thus, I tried to compute the X² of the values for each cluster. This X² was my tresh
#Yeah yeah yeah. I know, X² is a law, what I compute is just a value to compare to this law
#But I find this more pictured to talk about it this way
#PS: (don't trust this snippet, I think it's wrong)

partitions = []
tresh = 0

for lsa_part in [lsa1, lsa2]:
    # X² pour ne garder que les articles au dessus d'un certain seuil
    temp = 0
    sumConcept = 0
    for concept in transpose(lsa):
        for article in concept:
            sumConcept += article
            
            
            tresh += pow(article, 2)
    tresh /= (lsa_part.shape[0]*lsa_part.shape[1])
    

In [ ]:
def indexOf(matrixSrc, listSearched):
    if (len(matrixSrc[1]) != len(listSearched)):
        return -1
    pos = 0
    for l in matrixSrc:
        thesame = True
        for i in range(0, len(listSearched), 1):
            if l[i] != listSearched[i]:
                thesame = False
                break
        if thesame:
            return pos
        pos+=1
    return -2
a = [[1, 2, 3], [4, 2, 1], [2, 3, 5], [3, 2, 1]]
b = [3,2,1]
indexOf(a, b)

In [ ]:
#I know, transpose exists in numpy
#However, it didn't work with me because I got sparse matrix, so I was deseperate and I wrote my own function
def transpose(matrix):
    transp = []
    for j in range(0, matrix.shape[1], 1):
        l = []
        for i in range(0, matrix.shape[0], 1):
            l.append(matrix[i][j])
        transp.append(l)
    return transp


In [ ]:
def mostCommons(dico, tresh):
    res = []
    for name in dico:
        if dico[name] >= tresh:
            res.append(name)
    return res

test = {"abc": 3, "trucmuch":5, "doge":0}
mostCommons(test, 3)

In [ ]:
#IMPORTANT!! The good way to get an element from document-term matrix
tfidf.toarray().item(0, 0)